<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
cd Documents/GitHub/DS-Unit-4-Sprint-2-Neural-Networks/module4-Hyperparameter-Tuning/

/home/seek/Documents/GitHub/DS-Unit-4-Sprint-2-Neural-Networks/module4-Hyperparameter-Tuning


In [2]:
##### Your Code Here #####
import keras
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
# fix random seed for reproducibility
np.random.seed(38)

Using TensorFlow backend.


In [3]:
df = pd.read_csv('Telco-Customer-Churn.csv')


df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df = df.drop(columns=['customerID'])

In [5]:
print(df.shape)
df.head()

(7043, 20)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols : 
    df[i]  = df[i].replace({'No internet service' : 'No'})

In [7]:
df['gender'] = df['gender'].replace({'Male' : 0, 'Female' : 1})

In [8]:
binary_cols = [ 'Partner', 'Dependents', 'PhoneService','PaperlessBilling', 'Churn', 
               'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
               'StreamingTV', 'StreamingMovies']
for i in binary_cols : 
    df[i]  = df[i].replace({'Yes' : 1, 'No': 0})

In [9]:
df['MultipleLines'] = df['MultipleLines'].replace({'No phone service' : 0, 'No' : 1, 'Yes' : 2})

In [10]:
df = df.rename(columns={'MultipleLines' : 'Multiline Status:\n No phone service=0\n No=1\n Yes=2'})

In [11]:
df['InternetService'] = df['InternetService'].replace({'No': 0, 'DSL' : 1, 'Fiber optic' : 2})

In [12]:
df = df.rename(columns={'InternetService' : 'InternetService Type:\n None=0\n DSL=1\n Fiber Optic=2'})

In [13]:
df['Contract'] = df['Contract'].replace({'Month-to-month' : 0, 'One year' : 1, 'Two year' : 2})

In [14]:
df = df.rename(columns={'Contract' : 'Contract Type:\n None=0\n 1yr=1\n 2yr=2'})

In [15]:
df['PaymentMethod'] = df['PaymentMethod'].replace({'Mailed check' : 1, 'Electronic check' : 2,
                                                   'Bank transfer (automatic)' : 3, 'Credit card (automatic)' : 4})

In [16]:
df = df.rename(columns={'PaymentMethod' : 'PaymentMethod:\n Mailed Chk=1\n Electronic Chk=2\n Auto Bank=3\n Auto CC=4'})

In [17]:
df = df.rename(columns={'gender':'gender=M:0/F:1'})

In [18]:
df.head()

,gender=M:0/F:1,SeniorCitizen,Partner,Dependents,tenure,PhoneService,Multiline Status: No phone service=0 No=1 Yes=2,InternetService Type: None=0 DSL=1 Fiber Optic=2,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract Type: None=0 1yr=1 2yr=2,PaperlessBilling,PaymentMethod: Mailed Chk=1 Electronic Chk=2 Auto Bank=3 Auto CC=4,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1,2,29.85,29.85,0
1,0,0,0,0,34,1,1,1,1,0,1,0,0,0,1,0,1,56.95,1889.50,0
2,0,0,0,0,2,1,1,1,1,1,0,0,0,0,0,1,1,53.85,108.15,1
3,0,0,0,0,45,0,0,1,1,0,1,1,0,0,1,0,3,42.30,1840.75,0
4,1,0,0,0,2,1,1,2,0,0,0,0,0,0,0,1,2,70.70,151.65,1


In [19]:
import pandas_profiling 

In [20]:
pandas_profiling.ProfileReport(df)

Number of variables,20
Number of observations,7043
Total Missing (%),0.0%
Total size in memory,1.1 MiB
Average record size in memory,160.0 B
Numeric,6
Categorical,0
Boolean,13
Date,0
Text (Unique),0
Rejected,1


In [21]:
# MLP for Pima Indians Dataset with 10-fold cross validation
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold

# fix random seed for reproducibility
seed = 38
np.random.seed(seed)


# split into input (X) and output (y) variables
X = df.drop(columns=['Churn']).values
y = df['Churn'].values

# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, y):
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=19, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # Fit the model
  model.fit(X[train], y[train], epochs=150, batch_size=10, verbose=0)
  # evaluate the model
  scores = model.evaluate(X[test], y[test], verbose=0)
  print(f'{model.metrics_names[1]}: {(scores[1]*100):.2f}%') 
  cvscores.append(scores[1]*100)
print(f'{np.mean(cvscores):.2f}% +/- {np.std(cvscores):.2f}%')

W0719 09:37:49.871360 139761388300096 deprecation_wrapper.py:119] From /home/seek/anaconda3/envs/U4-S2-NNF/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0719 09:37:49.882289 139761388300096 deprecation_wrapper.py:119] From /home/seek/anaconda3/envs/U4-S2-NNF/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0719 09:37:49.884193 139761388300096 deprecation_wrapper.py:119] From /home/seek/anaconda3/envs/U4-S2-NNF/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0719 09:37:49.910994 139761388300096 deprecation_wrapper.py:119] From /home/seek/anaconda3/envs/U4-S2-NNF/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please

acc: 73.46%
acc: 73.46%
acc: 73.46%
acc: 73.46%
acc: 73.49%
73.46% +/- 0.01%


In [22]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 38
np.random.seed(seed)


# split into input (X) and output (y) variables
X = df.drop(columns=['Churn']).values
y = df['Churn'].values

# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=19, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

/home/seek/anaconda3/envs/U4-S2-NNF/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/20
4695/4695 [==============================] - 1s 214us/step - loss: 1.1488 - acc: 0.7014
Epoch 2/20
4695/4695 [==============================] - 1s 157us/step - loss: 0.6095 - acc: 0.7333
Epoch 3/20
4695/4695 [==============================] - 1s 158us/step - loss: 0.5902 - acc: 0.7333
Epoch 4/20
4695/4695 [==============================] - 1s 157us/step - loss: 0.5833 - acc: 0.7333
Epoch 5/20
4695/4695 [==============================] - 1s 159us/step - loss: 0.5810 - acc: 0.7333
Epoch 6/20
4695/4695 [==============================] - 1s 158us/step - loss: 0.5802 - acc: 0.7333
Epoch 7/20
4695/4695 [==============================] - 1s 157us/step - loss: 0.5800 - acc: 0.7333
Epoch 8/20
4695/4695 [==============================] - 1s 158us/step - loss: 0.5800 - acc: 0.7333
Epoch 9/20
4695/4695 [==============================] - 1s 163us/step - loss: 0.5800 - acc: 0.7333
Epoch 10/20
4695/4695 [==============================] - 1s 163us/step - loss: 0.5800 - acc: 0.7333
Epoch 11/

In [1]:
# define the grid search parameters
param_grid = {'batch_size': [80, 100],
              'epochs': [20, 40, 60]}

# Create Grid Search
grid = GridSearchCV(estimator=model, cv=10, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

NameError: name 'GridSearchCV' is not defined

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?